In [1]:
import pystac
from pathlib import Path
from shapely.geometry import Polygon, mapping
import rasterio as rio

import geopandas as gpd
from datetime import datetime


In [2]:
catalog = pystac.Catalog(
    id='forecast-test',
    description='This is a test of Landsat image forecasts',
    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [3]:
grid = gpd.read_file('./tmp/n_visits.geojson')

In [4]:
bbox = grid.total_bounds

In [5]:


def get_bbox_and_footprint(grid):

    # create the bounding box it will depend if it comes from rasterio or rioxarray
    bbox = grid.total_bounds

    # create the footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],
        [bbox[0], bbox[3]],
        [bbox[2], bbox[3]],
        [bbox[2], bbox[1]]
    ])

    return bbox, mapping(footprint)

In [6]:
grid.crs.to_string()

'EPSG:4326'

In [7]:
get_bbox_and_footprint(grid)

(array([-81.41094255, -55.61183   , -34.41094255,  13.38817   ]),
 {'type': 'Polygon',
  'coordinates': (((-81.41094255239946, -55.61183),
    (-81.41094255239946, 13.38817),
    (-34.41094255239946, 13.38817),
    (-34.41094255239946, -55.61183),
    (-81.41094255239946, -55.61183)),)})

In [8]:
import pandas as pd
from rasterio.warp import transform_bounds, transform_geom

# get a list of files
# folder = Path('./paranapanema/')
# folder = Path('./pystac/docs/example-catalog')
folder = Path('./tmp/')
collects = list(folder.rglob('*.geojson'))

for collect in collects:

    # open the mask with rasterio
    # ds = rio.open(col)
    col = gpd.read_file(collect)

    bbox, footprint = get_bbox_and_footprint(col)

    # Project to WGS84 to obtain in geometric coordinates
    geo_bounds = transform_bounds(col.crs.to_string(), 'EPSG:4326', *bbox)
    geo_footprint = transform_geom(col.crs.to_string(), 'EPSG:4326', footprint)

    # properties
    # idx = mask.stem[:38]
    # date = pd.to_datetime(mask.stem[11:26])
    # tile = mask.parts[1]

    item = pystac.Item(
        # id=idx,
        id = '1',
        geometry=geo_footprint,
        bbox=geo_bounds,
        # datetime=date,
        datetime = datetime.utcnow(),
        stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
        properties=dict(
            # tile=tile
        )
    )

    catalog.add_item(item)

print(len(list(catalog.get_items())))
catalog.describe()

1
* <Catalog id=forecast-test>
  * <Item id=1>


In [9]:
# ! git clone https://github.com/stac-utils/pystac.git

In [10]:
catalog.describe()

* <Catalog id=forecast-test>
  * <Item id=1>


In [12]:
catalog.normalize_hrefs('./stac_catalog')
catalog.save()